# Scatter points of the starting time in the progenitor parameter space

This notebook generates the scatter points of the starting time $t_{\text{start}}$ over progenitor parameter space, for a given $(\ell,|m|)$ harmonic. The analysis is perfomed with the $\texttt{London}$, $\texttt{Cheung}$, and $\texttt{TEOBPM}$ models.

**Imports & Settings**

In [17]:
# Scientific / numeric
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap

# Plot config
rcParams['text.usetex'] = False
rcParams['font.size']   = 28


**Ask user the $(\ell,|m|)$ mode to analyze**

As default, will choose $(\ell,m)=(2,2)$.

In [ ]:
# =======================================
# Select spherical-harmonic mode (ℓ,m)
# =======================================

available_modes = ["22", "21", "33", "32", "44"]
default_mode    = "22"

print("\nSelect spherical-harmonic mode $(\ell,m)$:")
for m in available_modes:
    print(f"  - {m}")

mode = input(f"Enter mode [{default_mode}]: ").strip() or default_mode

while mode not in available_modes:
    print("❌ Invalid choice. Please choose from the list above.")
    mode = input(f"Enter mode [{default_mode}]: ").strip() or default_mode

print(f"\n✅ Selected $(\ell,m)$ = {mode}")


Please choose $(l,m)$:
- 22
- 21
- 33
- 32
- 44

✅ Selected (l,m) = '33'


**Load mismatch results and NR simulations information**

Data are loaded directly from the GitHub repository.

* `avg_mismatches_all_times.npz`: contains, for the London, Cheung and TEOBPM models, the $(\ell,m)$ mismatch values for different starting times, where $(\ell,m)=\{(2,2),(2,1),(3,3),(3,2),(4,4)\}$.

* `SXS_BBH_nonprec_nonecc_all.txt`: to each simulation, we associate the triplet $(\eta,\chi_{+},\chi_{-})$.

In [27]:
# =======================================
# Load mismatch data + NR catalog
# =======================================

# Download data (if running fresh environment)
!wget -q -P files/ https://github.com/francesco-crescimbeni/Interpolating-function-of-ringdown-starting-time/raw/main/avg_mismatches_all_times.npz
!wget -q -P files/ https://github.com/francesco-crescimbeni/Interpolating-function-of-ringdown-starting-time/raw/main/SXS_BBH_nonprec_nonecc_all.txt

# ---------------------------------------
# Input files
# ---------------------------------------
npz_file     = "files/avg_mismatches_all_times.npz"
catalog_file = "files/SXS_BBH_nonprec_nonecc_all.txt"

# ---------------------------------------
# Load mismatch dictionary
# ---------------------------------------
data           = np.load(npz_file, allow_pickle=True)
avg_mismatches = data["avg_mismatches"].item()

# ---------------------------------------
# User-defined mismatch threshold
# ---------------------------------------
threshold = float(input("Enter mismatch threshold (0–1) [default=0.035]: ") or "0.035")

# ---------------------------------------
# Load NR metadata (η, χ+, χ-)
# ---------------------------------------
id_info = {}

with open(catalog_file, "r") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        
        fields = line.split()
        if len(fields) < 7:
            continue
        
        try:
            sim_id = fields[0]
            id_info[sim_id] = {
                "eta" : float(fields[2]),
                "chip": float(fields[5]),
                "chim": float(fields[6]),
            }
        except Exception:
            pass  # skip malformed entries

print(f"✅ Loaded {len(id_info)} NR simulations")
print(f"✅ Chosen mismatch threshold = {threshold}")


✅ Loaded 336 NR simulations
✅ Chosen mismatch threshold = 0.035


**Generate scatter plot**

This part of the notebook generates the scatter plots of $t_{\rm start}$ in the progenitor parameter space. For each point of the space, the starting time is interpolated by using the data on the `avg_mismatches` dictionary. 

For the simulations that don't reach the required mismatch value (i.e. no starting time could be extrapolated), we associate to it a white point with black boundary.

In [ ]:
# =======================================
# Prepare data and color map for 2D scatter plots
# =======================================

# ---------------------------------------
# Available models and example mode
# ---------------------------------------
available_models = ["KerrBinary_London", "KerrBinary_Cheung", "TEOBPM"]
example_model    = available_models[0]
example_mode     = available_modes[0]

# Retrieve all available ringdown start times (M units)
t_start_keys   = list(avg_mismatches[example_model][example_mode].keys())
starting_times = sorted(float(t) for t in t_start_keys)

# ---------------------------------------
# Custom diverging color map (blue → white → red)
# ---------------------------------------
colbBlue = "#4477AA"
colbRed  = "#EE6677"

def hex_to_rgb(hex_color):
    """Convert hex string to normalized RGB tuple."""
    hex_color = hex_color.lstrip("#")
    return tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (0, 2, 4))

def lighten_color(rgb_color, factor):
    """Blend RGB color with white by given factor."""
    return tuple(c + (1 - c) * factor for c in rgb_color)

blue_rgb = hex_to_rgb(colbBlue)
red_rgb  = hex_to_rgb(colbRed)
white    = (1, 1, 1)

colors = [
    blue_rgb,
    lighten_color(blue_rgb, 0.3),
    lighten_color(blue_rgb, 0.6),
    white,
    lighten_color(red_rgb, 0.6),
    lighten_color(red_rgb, 0.3),
    red_rgb,
]

cmap = LinearSegmentedColormap.from_list("blue_white_red", colors, N=256)

# ---------------------------------------
# Labels and figure config
# ---------------------------------------
titles     = [r"London", r"Cheung", r"TEOBPM"]
label_map  = {"eta": r"\eta", "chip": r"\chi_+", "chim": r"\chi_-"}

l_val      = mode[0]
m_val      = mode[1]
thresh_str = f"{round(threshold * 100, 1)}per"

print(f"\nProcessing mode {mode} with threshold {threshold*100:.2f}% ...")

# Refresh sorted start times (precaution if dict updated)
t_start_keys   = list(avg_mismatches[example_model][example_mode].keys())
starting_times = sorted(float(t) for t in t_start_keys)

# Create subplot grid: 3 planes (η–χ₊, η–χ₋, χ₊–χ₋) × models
fig, axs = plt.subplots(3, len(available_models), figsize=(8 * len(available_models), 13))
fig.subplots_adjust(right=0.88, hspace=0.4, wspace=0.3)

global_scatter = None  # handle for shared color bar

# =======================================
# Main plot loop over models
# =======================================
for col, model in enumerate(available_models):

    # Skip if model or mode does not exist in the data
    if model not in avg_mismatches or mode not in avg_mismatches[model]:
        continue

    success_records = []
    fail_records    = []

    # Loop through NR simulations from SXS catalog
    for sim_id, params in id_info.items():

        t_success = None  # earliest crossing time below threshold

        for t_start in starting_times:
            val = avg_mismatches.get(model, {}).get(mode, {}).get(t_start, {}).get(sim_id, np.nan)

            if val is not None and not np.isnan(val) and val > 0:
                if val <= threshold:
                    t_success = t_start
                    break

        if t_success is not None:
            success_records.append({
                "eta"    : params["eta"],
                "chip"   : params["chip"],
                "chim"   : params["chim"],
                "t_start": t_success,
            })
        else:
            fail_records.append({
                "eta"  : params["eta"],
                "chip" : params["chip"],
                "chim" : params["chim"],
            })

    df_success = pd.DataFrame(success_records)
    df_fail    = pd.DataFrame(fail_records)

    # ---------------------------------------
    # Axis plane combinations to plot
    # ---------------------------------------
    planes = [("eta", "chip"), ("eta", "chim"), ("chip", "chim")]

    for row, (x, y) in enumerate(planes):

        ax = axs[row][col]

        # Successful runs colored by t_start
        sc = ax.scatter(
            df_success[x], df_success[y],
            c=df_success["t_start"],
            cmap=cmap, vmin=0, vmax=30,
            s=60, linewidths=1
        )

        if global_scatter is None:
            global_scatter = sc

        # Failed runs = empty circle + X
        if not df_fail.empty:
            ax.scatter(
                df_fail[x], df_fail[y],
                facecolors="white", edgecolors="black",
                marker="o", s=60, linewidths=2
            )

        # Labels and title
        ax.set_xlabel(f"${label_map[x]}$", fontsize=22)
        ax.set_ylabel(f"${label_map[y]}$", fontsize=22)

        if row == 0:
            ax.set_title(f"{titles[col]}, $({l_val},{m_val})$", fontsize=20)

        # Grid styling
        ax.set_xticks(np.linspace(ax.get_xlim()[0], ax.get_xlim()[1], 9), minor=True)
        ax.set_yticks(np.linspace(ax.get_ylim()[0], ax.get_ylim()[1], 17), minor=True)
        ax.grid(True, which="major", ls=":")
        ax.tick_params(which="minor", length=4)

# ---------------------------------------
# Colorbar and save figure
# ---------------------------------------
cbar_ax = fig.add_axes([0.91, 0.15, 0.015, 0.7])
fig.colorbar(global_scatter, cax=cbar_ax).set_label(r"$t_{\rm start}~(M)$", fontsize=22)

pdf_path = f"plots/scatter2D_allmodels_lm{mode}_thresh_{thresh_str}.pdf"
plt.savefig(pdf_path)
plt.close()

print(f"✅ Saved: file://{pdf_path}")


Processing mode 33 with threshold 3.50% ...
✅ Saved: file://plots/scatter2D_allmodels_lm33_thresh_3.5per.pdf
